In [ ]:
import cv2
from matplotlib import pyplot
import numpy
from scipy.fftpack import dct
from scipy.fftpack import idct
import struct
import zlib

# Zadanie 1

In [ ]:
#
# PPM file header
#
ppm_ascii_header = 'P3 100 100 255 '# TODO: implement
ppm_binary_header ='P6 100 100 255 ' # TODO: implement

#
# Image data
#
#image = numpy.array([], dtype=numpy.uint8)  # TODO: implement
image = numpy.zeros((100, 100, 3)).astype('uint8')
image[:] = [0, 255, 255]


# Save the PPM image as an ASCII file
#
with open('lab4-ascii.ppm', 'w') as fh:
    fh.write(ppm_ascii_header)
    image.tofile(fh, sep=' ')
    fh.write('\n')

#
# Save the PPM image as a binary file
#
with open('lab4-binary.ppm', 'wb') as fh:
    fh.write(bytearray(ppm_binary_header, 'ascii'))
    image.tofile(fh)

In [ ]:
#
# Display image
#
image_from_file = cv2.imread('lab4-ascii.ppm')
pyplot.imshow(cv2.cvtColor(image_from_file, cv2.COLOR_BGR2RGB))

In [ ]:
#
# Display image
#
image_from_file = cv2.imread('lab4-binary.ppm')
pyplot.imshow(cv2.cvtColor(image_from_file, cv2.COLOR_BGR2RGB))

# Zadanie 2

In [ ]:
ppm_ascii_header = 'P3 1792 128 255\n'
width, height = map(int, ppm_ascii_header.split()[1:3])

rainbow = numpy.zeros((height, width, 3)).astype('uint8')
color = [0, 0, 0]
for i in range(0, 7*256):
    rainbow[:128,i,:] = color[0:3]
    if i < 255:
        color = numpy.add(color,[0, 0, 1])
    elif 255 <= i < 2*255:
        color = numpy.add(color,[0, 1, 0])
    elif 2*255 <= i < 3*255:
        color = numpy.add(color,[0, 0, -1])
    elif 3*255 <= i < 4*254:
        color = numpy.add(color,[1, 0, 0])
    elif 4*255 < i <= 5*255:
        color = numpy.add(color,[0, -1, 0])
    elif 5*255 < i <= 6*255:
        color = numpy.add(color,[0, 0, 1])
    elif 6*255 < i <= 7*255:
        color = numpy.add(color,[0, 1, 0])
    
    
with open('lab4-rainbow.ppm', 'w') as fh:
    fh.write(ppm_ascii_header)
    rainbow.tofile(fh, sep=' ')
    fh.write('\n')

In [ ]:
#
# Display image
#
image_from_file = cv2.imread('lab4-rainbow.ppm')
pyplot.imshow(cv2.cvtColor(image_from_file, cv2.COLOR_BGR2RGB))

# Zadanie 3

In [ ]:
#
# Image data
#

image = rainbow  # TODO: implement

#
# Construct signature
#
png_file_signature = b'\x89PNG\r\n\x1a\n'  # TODO: implement

# Construct header
#
header_id = b'IHDR'  # TODO: implement
header_content = bytes.fromhex('00000700000000800802000000')   # TODO: implement
header_size = struct.pack('!I', len(header_content))  # TODO: implement
header_crc = struct.pack('!I', zlib.crc32(header_id + header_content))  # TODO: implement
    
png_file_header = header_size + header_id + header_content + header_crc


#
# Construct data
#

data_id = b'IDAT'  # TODO: implement
data_content = zlib.compress(b''.join([b'\x00' + bytes(row) for row in image])) 
data_size = struct.pack('!I', len(data_content))  # TODO: implement
data_crc = struct.pack('!I', zlib.crc32(data_id + data_content))  # TODO: implement
png_file_data = data_size + data_id + data_content + data_crc

#
# Consruct end
#
end_id = b'IEND'
end_content = b''
end_size = struct.pack('!I', len(end_content))
end_crc = struct.pack('!I', zlib.crc32(end_id + end_content))
png_file_end = end_size + end_id + end_content + end_crc


#
# Save the PNG image as a binary file
#
with open('lab4.png', 'wb') as fh:
    fh.write(png_file_signature)
    fh.write(png_file_header)
    fh.write(png_file_data)
    fh.write(png_file_end)

In [ ]:
#
# Display image
#
image_from_file = cv2.imread('lab4.png')
pyplot.imshow(cv2.cvtColor(image_from_file, cv2.COLOR_BGR2RGB))

# Funkcje pomocnicze do zadań 4 i 5

In [ ]:
#
# 2d Discrete Cosinus Transform
#
def dct2(array):
    return dct(dct(array, axis=0, norm='ortho'), axis=1, norm='ortho')


def idct2(array):
    return idct(idct(array, axis=0, norm='ortho'), axis=1, norm='ortho')


#
# Calculate quantisation matrices
#
# Based on: https://www.hdm-stuttgart.de/~maucher/Python/MMCodecs/html/jpegUpToQuant.html
#           #step-3-and-4-discrete-cosinus-transform-and-quantisation
#
_QY = numpy.array([[16, 11, 10, 16, 24, 40, 51, 61],
                   [12, 12, 14, 19, 26, 48, 60, 55],
                   [14, 13, 16, 24, 40, 57, 69, 56],
                   [14, 17, 22, 29, 51, 87, 80, 62],
                   [18, 22, 37, 56, 68, 109, 103, 77],
                   [24, 35, 55, 64, 81, 104, 113, 92],
                   [49, 64, 78, 87, 103, 121, 120, 101],
                   [72, 92, 95, 98, 112, 100, 103, 99]])

_QC = numpy.array([[17, 18, 24, 47, 99, 99, 99, 99],
                   [18, 21, 26, 66, 99, 99, 99, 99],
                   [24, 26, 56, 99, 99, 99, 99, 99],
                   [47, 66, 99, 99, 99, 99, 99, 99],
                   [99, 99, 99, 99, 99, 99, 99, 99],
                   [99, 99, 99, 99, 99, 99, 99, 99],
                   [99, 99, 99, 99, 99, 99, 99, 99],
                   [99, 99, 99, 99, 99, 99, 99, 99]])


def _scale(QF):
    if QF < 50 and QF >= 1:
        scale = numpy.floor(5000 / QF)
    elif QF < 100:
        scale = 200 - 2 * QF
    else:
        raise ValueError('Quality Factor must be in the range [1..99]')

    scale = scale / 100.0
    return scale


def QY(QF=85):
    return _QY * _scale(QF)


def QC(QF=85):
    return _QC * _scale(QF)

def zig_zag_index(k, n):
    # upper side of interval
    if k >= n * (n + 1) // 2:
        i, j = zig_zag_index(n * n - 1 - k, n)
        return n - 1 - i, n - 1 - j
    # lower side of interval
    i = int((np.sqrt(1 + 8 * k) - 1) / 2)
    j = k - i * (i + 1) // 2
    return (j, i - j) if i & 1 else (i - j, j)

# Zadania 4 i 5

In [ ]:
#
# 0. Image data
#
# TODO: implement (zad. 4)
image = rainbow
#
# 1. Convert RGB to YCbCr
#
# TODO: implement (zad. 4)
image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
#
# 2. Downsampling on Cb and Cr channels
#
# TODO: implement (zad. 4)
scale = 4   # level of downsampling
scale_helper = {2:[1,2], 4:[2,2], 1:[1,1]} 
height, width = scale_helper[scale]

Y = image[:,:,0]
Cr_Cb = image[::height, ::width, 1:]

#
# 3. Produce 8x8 blocks
#
# TODO: implement (zad. 4)

Y_blocks =  [[0 for x in range(image.shape[0]//8)] for y in range(image.shape[1]//8)]
for y in range(0, image.shape[0]//8):
    for x in range(0, image.shape[1]//8):
        Y_blocks[x][y] = Y[8*y:8*(y+1),8*x:8*(x+1)]
        
Cr_blocks =  [[0 for x in range(Cr_Cb.shape[0]//8)] for y in range(Cr_Cb.shape[1]//8)]
Cb_blocks =  [[0 for x in range(Cr_Cb.shape[0]//8)] for y in range(Cr_Cb.shape[1]//8)]
for y in range(0, Cr_Cb.shape[0]//8):
    for x in range(0, Cr_Cb.shape[1]//8):
        Cr_blocks[x][y] = Cr_Cb[8*y:8*(y+1),8*x:8*(x+1),0]        
        Cb_blocks[x][y] = Cr_Cb[8*y:8*(y+1),8*x:8*(x+1),1]        

#
# 4. Calculate DCT on each block
#
# TODO: implement (zad. 5)
Y_dct, Cr_dct, Cb_dct = Y_blocks, Cr_blocks, Cb_blocks
for y in range(0, image.shape[0]//8):
    for x in range(0, image.shape[1]//8):
        Y_dct[x][y] = dct2(Y_blocks[x][y])
for y in range(0, Cr_Cb.shape[0]//8):
    for x in range(0, Cr_Cb.shape[1]//8):
        Cr_dct[x][y] = dct2(Cr_blocks[x][y])
        Cb_dct[x][y] = dct2(Cb_blocks[x][y])
#
# 5. Divide each block by quantisation matrix
#
# TODO: implement (zad. 5)
Y_divided, Cr_divided, Cb_divided = Y_blocks, Cr_blocks, Cb_blocks
for y in range(0, image.shape[0]//8):
    for x in range(0, image.shape[1]//8):
        Y_divided[x][y] = Y_dct[x][y] / QY()
for y in range(0, Cr_Cb.shape[0]//8):
    for x in range(0, Cr_Cb.shape[1]//8):
        Cr_divided[x][y] = Cr_dct[x][y] / QC()        
        Cb_divided[x][y] = Cb_dct[x][y] / QC()       

        
#
# 6. Round values in each block to integers
#
# TODO: implement (zad. 5)

Y_round, Cr_round, Cb_round = Y_blocks, Cr_blocks, Cb_blocks
for y in range(0, image.shape[0]//8):
    for x in range(0, image.shape[1]//8):
        Y_round = numpy.rint(Y_divided[x][y])
for y in range(0, Cr_Cb.shape[0]//8):
    for x in range(0, Cr_Cb.shape[1]//8):
        Cr_round[x][y] = numpy.rint(Cr_divided[x][y])      
        Cb_round[x][y] = numpy.rint(Cb_divided[x][y]) 
#
# 7. Zig Zag
#
# TODO: implement (zad. 4)
Y_zigzags=[]
Cr_zigzags=[]
Cb_zigzags=[]

for y in range(image.shape[0]//8):
    for x in range(image.shape[1]//8):
        block = Y_divided[x][y]
        Y_zigzags.append(numpy.concatenate([numpy.diagonal(block[::-1,:], 
                                                     k)[::(2*(k % 2)-1)] for k in range(1-block.shape[0], block.shape[0])]))

for y in range(Cr_Cb.shape[0]//8):
    for x in range(Cr_Cb.shape[1]//8):
        Cr_block = Cr_divided[x][y]
        Cb_block = Cb_divided[x][y]
        Cr_zigzags.append(numpy.concatenate([numpy.diagonal(Cr_block[::-1,:], 
                                                k)[::(2*(k % 2)-1)] for k in range(1-Cr_block.shape[0], Cr_block.shape[0])]))
        Cb_zigzags.append(numpy.concatenate([numpy.diagonal(Cb_block[::-1,:], 
                                                k)[::(2*(k % 2)-1)] for k in range(1-Cb_block.shape[0], Cb_block.shape[0])]))
        
        

#
# 8. Flatten, concatenate, compress and calculate the size -- how many bytes?
#    without downsampling - 85347, every 2 - 56883 , every 42654  
#
#
# TODO: implement (zad. 4)
Y_ready = numpy.concatenate(Y_zigzags).ravel()
Cr_ready = numpy.concatenate(Cr_zigzags).ravel()
Cb_ready = numpy.concatenate(Cb_zigzags).ravel()
data = numpy.concatenate((Y_ready, Cr_ready, Cb_ready))

final = zlib.compress(b''.join([bytes(row) for row in data])) 

#
# 7'. Undo Zig Zag
#
# We can skip it in this exercise! We did Zig Zag only for analysis in step 8.
# You can continue with result from step 6. instead of implementing undo here.
#

#
# 6'. Nothing to do here   ¯\_(ツ)_/¯
#
# No conversion is really needed here, just proceed to the next step.
#

#
# 5'. Reverse division by quantisation matrix -- multiply
#
# TODO: implement (zad. 5)
Y_rev_div, Cr_rev_div, Cb_rev_div = Y_blocks, Cr_blocks, Cb_blocks
for y in range(0, image.shape[0]//8):
    for x in range(0, image.shape[1]//8):
        Y_rev_div[x][y] = Y_divided[x][y] * QY()
for y in range(0, Cr_Cb.shape[0]//8):
    for x in range(0, Cr_Cb.shape[1]//8):
        Cr_rev_div[x][y] = Cr_divided[x][y] * QC()        
        Cb_rev_div[x][y] = Cb_divided[x][y] * QC()
#
# 4'. Reverse DCT
#
# TODO: implement (zad. 5)
Y_idct, Cr_idct, Cb_idct = Y_dct, Cr_dct, Cb_dct
for y in range(0, image.shape[0]//8):
    for x in range(0, image.shape[1]//8):
        Y_idct[x][y] = idct2(Y_dct[x][y])
for y in range(0, Cr_Cb.shape[0]//8):
    for x in range(0, Cr_Cb.shape[1]//8):
        Cr_idct[x][y] = idct2(Cr_dct[x][y])
        Cb_idct[x][y] = idct2(Cb_dct[x][y])
#
# 3'. Combine 8x8 blocks to original image
#
# TODO: implement (zad. 4)

Y_after = numpy.zeros((image.shape[0], image.shape[1])).astype('uint8')
Cr_Cb_after = numpy.zeros((Cr_Cb.shape[0], Cr_Cb.shape[1],2)).astype('uint8')
for y in range(0, image.shape[0]//8):
    for x in range(0, image.shape[1]//8):
        Y_after[8*y:8*(y+1),8*x:8*(x+1)] = Y_blocks[x][y]
for y in range(0, Cr_Cb.shape[0]//8):
    for x in range(0, Cr_Cb.shape[1]//8):
        Cr_Cb_after[8*y:8*(y+1),8*x:8*(x+1), 0] = Cr_blocks[x][y]
        Cr_Cb_after[8*y:8*(y+1),8*x:8*(x+1), 1] = Cb_blocks[x][y]

#
# 2'. Upsampling on Cb and Cr channels
#
# TODO: implement (zad. 4)
upscaled = numpy.zeros([image.shape[0], image.shape[1],3]).astype('uint8')
for y in range(height):
    for x in range(width):
        upscaled[y::height, x::width, 1:] = Cr_Cb_after[:,:,:]

upscaled[:,:,0] = Y_after

#
# 1'. Convert YCbCr to RGB
#
# TODO: implement (zad. 4)

image = cv2.cvtColor(upscaled, cv2.COLOR_YCrCb2RGB)
#
# 0'. Save the decoded image -- as PPM or PNG
#
# TODO: implement (zad. 4)
with open('lab4-po-kompresji-i-dekompresji-jpeg.ppm', 'w') as fh:
    fh.write(ppm_ascii_header)
    image.tofile(fh, sep=' ')
    fh.write('\n')

In [ ]:
#
# Display image
#
image_from_file = cv2.imread('lab4-po-kompresji-i-dekompresji-jpeg.ppm')
pyplot.imshow(cv2.cvtColor(image_from_file, cv2.COLOR_BGR2RGB))



# Obserwacje z zadań 4 i 5

Zadanie 4 - im większy stopień samplowania tym mniej waży wynikowy obraz. 
Mimo down amplingu różnice w obrazie nie są znaczące, sprawdzałem też dla przesadnie dużych wartości
i dopiero wtedy różnica była znacząca. Po przybliżeniu widać efekty samplowania ale w standardowych przypadkach bez
"ekstremalnych" przybliżeń, różnica jest do przyjęcia..
W skrócie W4rto - bo zyskujemy sporo miejsca przy niewielkiej utracie jakości zdjęcia.

Zadanie 5 - Im mniejsza wartość czynnika QF tym obraz mniej waży (z tym że różnica nie jest aż tak duża jak przy 
downsamplingu). Przy mniejszych wartościach widoczne są "schodki" kolorów - obraz przypomina układankę z pasków, a nie 
"gładkie" przejście po przestrzeni barw. Przy "eksperymentach" i ustawieniu QF skrajnie tj.: 99 dla składowych C, 
a 1 dla Y obraz był stosunkowo bliski oryginalnemu, a przejście między barwami było dość płynne. 
